In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,GlobalAveragePooling2D, Concatenate, Reshape,GlobalMaxPooling2D, Activation, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import IPython.display as display
from PIL import Image
import numpy as np
import pandas
import matplotlib.pyplot as plt
import pathlib
import datetime
import math
import sys
import neptune
# GPU setup
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True) 


xrayPredictor
xrayPredictor-448x448
xrayPredictor-custom

In [4]:
project = 'lassegoransson/xrayPredictor'
import neptune
key = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiOWUxN2YwMTUtNjM1Ny00NmVlLWIzOTctNzAwYTllMGNmMTg2In0="
project = neptune.init(project,api_token=key)

In [16]:
name = "XRAY-49"
modelname = "/home/lasg/bachelor-data/models/"+name+".hdf5"
model = tf.keras.models.load_model(modelname, custom_objects={'Activation': tf.keras.layers.Activation})

In [10]:
test_df = pandas.read_csv("/home/lasg/bachelor-data/allTest.csv")
test_datagen = ImageDataGenerator(
        rescale=1./255,
        )

shape = model.input_shape
height = 224
width = 224
imgdir = "/home/lasg/bachelor-data/data_448x448/"


if model.input_shape[3] == 1:
    
    test_generator = test_datagen.flow_from_dataframe(
            dataframe=test_df,
            directory=imgdir,
            x_col="filename",
            y_col='label',
            target_size=(height, width),
            batch_size=1,
            shuffle=False,
            class_mode="raw",
            color_mode="grayscale"
            #color_mode="rgb"
            )
else: 
        test_generator = test_datagen.flow_from_dataframe(
            dataframe=test_df,
            directory=imgdir,
            x_col="filename",
            y_col='label',
            target_size=(height, width),
            batch_size=1,
            shuffle=False,
            class_mode="raw",
            #color_mode="grayscale"
            color_mode="rgb"
            )
        

Found 806 validated image filenames.


# Evaluate




In [17]:
labels = []
predicts = []
n = []
i=0
for b in range(0,test_generator.n):
  img,y= test_generator.next()
  img=img[0]
  sys.stdout.write("\r" + str(100*i/test_generator.n))
  label = y[0]
  predict = model.predict(np.expand_dims(img, axis=0))[0][0]
  predicts.append(predict)
  labels.append(label)
  val =(1-(label/predict))
  n.append(val)
  i+=1

df = pandas.DataFrame(data={"pdiviate": n, "label" : labels, "predicts": predicts})
df.to_csv("/home/lasg/bachelor-data/models/predictions/"+name+".csv", sep=',',index=False)

df
  

99.8759305210918266

,pdiviate,label,predicts
0,-0.231659,0.754,0.612182
1,-0.240748,0.990,0.797906
2,-0.236878,0.754,0.609599
3,-0.291864,0.303,0.234545
4,-0.220551,0.890,0.729179
...,...,...,...
801,-0.246418,0.990,0.794276
802,-0.751945,0.227,0.129570
803,-0.220754,0.598,0.489861
804,-0.025008,0.754,0.735604


# Model Architecture

In [9]:
modelname.split("/")[1]

'model_resnet_aspect_baseline_3layer_l2reg.py_checkpoint-515x320-016-0.0029903836548328.hdf5'

In [10]:
tf.keras.utils.plot_model(model, show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


# SCORE

In [11]:
print("")
print("MSE: "+str(mse))
print("")
print("Deviation from true weight (< 5% = 90 is project goal)")
print("< 5.0% = " +  str(100*hits[0]/(test_generator.n))+ "%")
print("< 10.0% = " + str(100*hits[1]/(test_generator.n))+ "%")
print("< 15.0% = " + str(100*hits[2]/(test_generator.n))+ "%")
print("< 20.0% = " + str(100*hits[3]/(test_generator.n))+ "%")
print("< 25.0% = " + str(100*hits[4]/(test_generator.n))+ "%")
print("< 30.0% = " + str(100*hits[5]/(test_generator.n))+ "%")
print("")
print("Deviation summed over full batch")
print("Predicted weight: "+str(labelval)+" True weight: "+str(labeltrueVal)+" Percentage: "+str((labelval/labeltrueVal)*100))


MSE: 0.002990381483630136

Deviation from true weight (< 5% = 90 is project goal)
< 5.0% = 53.225806451612904%
< 10.0% = 84.61538461538461%
< 15.0% = 95.16129032258064%
< 20.0% = 98.75930521091811%
< 25.0% = 99.75186104218362%
< 30.0% = 99.87593052109182%

Deviation summed over full batch
Predicted weight: 521.4762737900019 True weight: 534.0040000000014 Percentage: 97.65400142882834


In [12]:
ex.log_metric("5%_score",100*hits[0]/(test_generator.n))
ex.log_metric("10%_score",100*hits[1]/(test_generator.n))
ex.log_metric("15%_score",100*hits[2]/(test_generator.n))
ex.log_metric("20%_score",100*hits[3]/(test_generator.n))
ex.log_metric("25%_score",100*hits[4]/(test_generator.n))
ex.log_metric("30%_score",100*hits[5]/(test_generator.n))

# Residuals

In [ ]:
plt.figure(figsize=(20,10))
plt.stem(range(0,len(n)),nnon,use_line_collection=True)
plt.title('Model accuracy')
plt.ylabel('Accuracy (%)')
plt.xlabel('N of test')
plt.show()